# Reinforcement Learning - Monte Carlo
If you want to test/submit your solution **restart the kernel, run all cells and submit the mc_autograde.py file into codegrade.**

In [ ]:
# This cell imports %%execwritefile command (executes cell and writes it into file). 
# All cells that start with %%execwritefile should be in mc_autograde.py file after running all cells.
from custommagics import CustomMagics
get_ipython().register_magics(CustomMagics)

In [ ]:
%%execwritefile mc_autograde.py
import numpy as np
from collections import defaultdict
from tqdm import tqdm as _tqdm

def tqdm(*args, **kwargs):
    return _tqdm(*args, **kwargs, mininterval=1)  # Safety, do not overflow buffer

In [ ]:
import matplotlib.pyplot as plt
import sys


%matplotlib inline

assert sys.version_info[:3] >= (3, 6, 0), "Make sure you have Python 3.6 installed!"

## 1. Monte Carlo Prediction

For the Monte Carlo Prediction we will look at the Blackjack game (Example 5.1 from the book), for which the `BlackjackEnv` is implemented in `blackjack.py`. Note that compared to the gridworld, the state is no longer a single integer, which is why we use a dictionary to represent the value function instead of a numpy array. By using `defaultdict`, each state gets a default value of 0.

In [ ]:
from blackjack import BlackjackEnv
env = BlackjackEnv()

For the Monte Carlo algorithm, we no longer have transition probabilities and we need to *interact* with the environment. This means that we start an episode by using `env.reset` and send the environment actions via `env.step` to observe the reward and next observation (state).

In [ ]:
# So let's have a look at what we can do in general with an environment...
import gym
?gym.Env

In [ ]:
# We can also look at the documentation/implementation of a method
?env.step

In [ ]:
?gym.spaces

In [ ]:
??BlackjackEnv

A very simple policy for Blackjack is to *stick* if we have 20 or 21 points and *hit* otherwise. We want to know how good this policy is. This policy is *deterministic* and therefore a function that maps an observation to a single action. Technically, we can implement this as a dictionary , a function or a class with a function, where we use the last option. Moreover, it is often useful (as you will see later) to implement a function that returns  the probability $\pi(a|s)$ for the state action pair (the probability that this policy would perform certain action in given state). We group these two functions in a policy class. To get started, let's implement this simple policy for BlackJack.

In [ ]:
%%execwritefile -a mc_autograde.py

class SimpleBlackjackPolicy(object):
    """
    A simple BlackJack policy that sticks with 20 or 21 points and hits otherwise.
    """
    def get_probs(self, states, actions):
        """
        This method takes a list of states and a list of actions and returns a numpy array that contains a probability
        of perfoming action in given state for every corresponding state action pair. 

        Args:
            states: a list of states.
            actions: a list of actions.

        Returns:
            Numpy array filled with probabilities (same length as states and actions)
        """
        # YOUR CODE HERE
        
        # So we need to determine for every input state-action pair, what the resulting policy distribution is
        # This means that the input will be a single state and a single action per index. 
        # We then need to determine if, according to our policy, the action should be taken (prob=1) 
        # or not (prob=0)
        
        # state is a tuple of (player's current sum, dealer's single showing card, boolean for usable ace)
        probs = []
        for index, (state, action) in enumerate(zip(states, actions)):
            chosen_action = self.sample_action(state)
            if action == chosen_action:
                probs.append(1)
            else:
                probs.append(0)
                    
        
        return np.array(probs)
    
    def sample_action(self, state):
        """
        This method takes a state as input and returns an action sampled from this policy.  

        Args:
            state: current state

        Returns:
            An action (int).
        """
        # YOUR CODE HERE
        if state[0] == 20 or state[0] == 21: #Now we should stick (0)
            action = 0
        else: # Otherwise hit
            action = 1
    
        return action

In [ ]:
# Let's check if it makes sense
env = BlackjackEnv()
s = env.reset()
policy = SimpleBlackjackPolicy()
print("State: {}\nSampled Action: {}\nProbabilities [stick, hit]: {}".format(s, policy.sample_action(s), policy.get_probs([s,s],[0,1])))

Since there are multiple algorithms which require data from single episode (or multiple episodes) it is often useful to write a routine that will sample a single episode. This will save us some time later. Implement a *sample_episode* function which uses environment and policy to sample a single episode.

In [ ]:
%%execwritefile -a mc_autograde.py

def sample_episode(env, policy):
    """
    A sampling routine. Given environment and a policy samples one episode and returns states, actions, rewards
    and dones from environment's step function and policy's sample_action function as lists.

    Args:
        env: OpenAI gym environment.
        policy: A policy which allows us to sample actions with its sample_action method.

    Returns:
        Tuple of lists (states, actions, rewards, dones). All lists should have same length. 
        Hint: Do not include the state after the termination in the list of states.
    """
    states = []
    actions = []
    rewards = []
    dones = []
    
    # YOUR CODE HERE
    done = False
    state = env.reset() # Could also use env._get_obs(), but codegrade seems to expect this
    while done == False:
        states.append(state)
    
        action = policy.sample_action(state)
        actions.append(action)
    
        state, reward, done, _ = env.step(action)
        
        rewards.append(reward)
        dones.append(done)

    return states, actions, rewards, dones

In [ ]:
# Let's sample some episodes
env = BlackjackEnv()
policy = SimpleBlackjackPolicy()
for episode in range(3):
    trajectory_data = sample_episode(env, policy)
    print("Episode {}:\nStates {}\nActions {}\nRewards {}\nDones {}\n".format(episode,*trajectory_data))

Now implement the MC prediction algorithm (either first visit or every visit). Hint: you can use `for i in tqdm(range(num_episodes))` to show a progress bar. Use the sampling function from above to sample data from a single episode.

In [ ]:
%%execwritefile -a mc_autograde.py

from collections import Counter
import gym

def mc_prediction(env, policy, num_episodes, discount_factor=1.0, sampling_function=sample_episode):
    """
    Monte Carlo prediction algorithm. Calculates the value function
    for a given policy using sampling.
    
    Args:
        env: OpenAI gym environment.
        policy: A policy which allows us to sample actions with its sample_action method.
        num_episodes: Number of episodes to sample.
        discount_factor: Gamma discount factor.
        sampling_function: Function that generates data from one episode.
    
    Returns:
        A dictionary that maps from state -> value.
        The state is a tuple and the value is a float.
    """

    # Keeps track of current V and count of returns for each state
    # to calculate an update.
    V = defaultdict(float)
    returns_count = defaultdict(float)
    
    # YOUR CODE HERE    
        
    # Due to the structure of the gym environment, it is not trivial to map the entire state space
    # so we only map the state space of the BlackJack env
    count_zeros = False
    if (isinstance(env.observation_space, gym.spaces.tuple_space.Tuple)):
        if (len(env.observation_space.spaces) == 3):
            count_zeros = True
        
        
    state_tuples = [(first, second, bool(third)) for first in range(2,env.observation_space.spaces[0].n)
                       for second in range(1,env.observation_space.spaces[1].n)
                       for third in range(env.observation_space.spaces[2].n)]
    returns = {state_tuple: [] for state_tuple in state_tuples}
    
    if count_zeros:
        # Replace the returns_count with a Counter object, and fill with all possible states
        returns_count = Counter({state_tuple: 0 for state_tuple in state_tuples})
    
    
    for episode in tqdm(range(num_episodes)): # num_episodes
        
        env.reset()
        states, actions, rewards, dones = sampling_function(env, policy)
        p_return = 0
        
        for index in reversed(range(len(states))): # Reverse so we loop in opposite direction through timesteps
            c_state = states[index]
            c_action = actions[index]
            c_reward = rewards[index]

            p_return = discount_factor * p_return + c_reward
                        
            if len(returns[c_state]) == 0:
                returns[c_state] = [p_return]
            else:
                returns[c_state].append(p_return)
            
            if count_zeros:
                returns_count[c_state] += 1
    
    V = {state: np.nan_to_num(np.mean(value)) for (state, value) in returns.items()}
    if count_zeros:
        zero_counts = [True for item in list(returns_count) if returns_count[item] == 0]
    
        no_of_zero = sum(zero_counts)
        if no_of_zero>0:
            print(f"Did not reach {no_of_zero} states in MC estimation. Value estimation for these states is missing.")
        else:
            print("Reached all states in MC estimation.")
        
    return V

In [ ]:
V_10k = mc_prediction(env, SimpleBlackjackPolicy(), num_episodes=10000)
print(V_10k)

Now make *4 plots* like Figure 5.1 in the book. You can either make 3D plots or heatmaps. Make sure that your results look similar to the results in the book. Give your plots appropriate titles, axis labels, etc.

In [ ]:
%%time
# Let's run your code one time
V_10k = mc_prediction(env, SimpleBlackjackPolicy(), num_episodes=10000)
V_500k = mc_prediction(env, SimpleBlackjackPolicy(), num_episodes=500000)

In [ ]:
# YOUR CODE HERE
from mpl_toolkits.mplot3d import Axes3D

def get_mesh(value_dict, usable=True, plot_surf=False, ax=None):
    
    value_array = np.array([[state[0], state[1], value_dict[state]] for state in value_dict.keys() if state[2]==usable])
    
    max_y = int(np.max(value_array[:, 0])) # y axis for player sum
    min_y = int(np.min(value_array[:, 0]))
    max_x = int(np.max(value_array[:, 1])) # x axis for dealer value
    min_x = int(np.min(value_array[:, 1]))
    range_x = range(min_x, max_x+1)
    range_y = range(min_y, max_y+1)
    X, Y = np.meshgrid(range_x, range_y)
    Z = np.zeros(X.shape)
    
    for y, x, value in value_array:
        Z[int(y)-min_y, int(x)-min_x] = value
        
    
    
    if plot_surf:
        if ax is None:
            ax = plt.axes(projection='3d')
        ax.plot_wireframe(X, Y, Z)
        ax.set_ylim3d(12, 21)
        ax.set_xlim3d(1,10)
        ax.set_zlim3d(-1,1)
        
        return X, Y, Z, ax
    else:
        return X, Y, Z


for iteration_index, V_dict in enumerate([V_10k, V_500k]):
    for usable_index, usable in enumerate([True, False]):
        ax = plt.subplot(2, 2, 1+iteration_index + 2*usable_index, projection='3d')
        X, Y, Z, ax = get_mesh(V_dict, usable=usable, plot_surf=True, ax=ax)
        if usable:
            if iteration_index == 0:
                ax.set_title("Usable ace at 10k episodes")
            else:
                ax.set_title("Usable ace at 500k episodes")
        else:
            if iteration_index == 0:
                ax.set_title("No usable ace at 10k episodes")
            else:
                ax.set_title("No usable ace at 500k episodes")
        
        


## 2. Off-policy Monte Carlo prediction
In real world, it is often beneficial to learn from the experience of others in addition to your own. For example, you can probably infer that running off the cliff with a car is a bad idea if you consider what "return" people who have tried it received.

Similarly, we can benefit from the experience of other agents in reinforcement learning. In this exercise we will use off-policy monte carlo to estimate the value function of our target policy using the experience from a different behavior policy. Our target policy will be the simple policy defined above (stick if we have *20* or *21* points) and we will use a random policy that randomly chooses to stick or hit (both with 50% probability) as a behavior policy. As a first step, implement a random BlackJack policy.

In [ ]:
%%execwritefile -a mc_autograde.py

class RandomBlackjackPolicy(object):
    """
    A random BlackJack policy.
    """
    def get_probs(self, states, actions):
        """
        This method takes a list of states and a list of actions and returns a numpy array that contains 
        a probability of perfoming action in given state for every corresponding state action pair. 

        Args:
            states: a list of states.
            actions: a list of actions.

        Returns:
            Numpy array filled with probabilities (same length as states and actions)
        """
        # YOUR CODE HERE
        
        probs = np.ones(len(states))/2
        return probs
    
    def sample_action(self, state):
        """
        This method takes a state as input and returns an action sampled from this policy.  

        Args:
            state: current state

        Returns:
            An action (int).
        """
        # YOUR CODE HERE
        
        action = np.random.choice(1)
        
        
        return action


In [ ]:
# Let's check if it makes sense
env = BlackjackEnv()
s = env.reset()
policy = RandomBlackjackPolicy()
print("State: {}\nSampled Action: {}\nProbabilities [stick, hit]: {}".format(s, policy.sample_action(s), policy.get_probs([s,s],[0,1])))

Now implement the MC prediction algorithm with ordinary importance sampling. Use the sampling function from above to sample data from a single episode.

Hint: Get probs functions may be handy. You can use `for i in tqdm(range(num_episodes))` to show a progress bar.

In [ ]:
%%execwritefile -a mc_autograde.py
import gym

def mc_importance_sampling(env, behavior_policy, target_policy, num_episodes, discount_factor=1.0,
                           sampling_function=sample_episode):
    """
    Monte Carlo prediction algorithm. Calculates the value function
    for a given target policy using behavior policy and ordinary importance sampling.
    
    Args:
        env: OpenAI gym environment.
        behavior_policy: A policy used to collect the data.
        target_policy: A policy which value function we want to estimate.
        num_episodes: Number of episodes to sample.
        discount_factor: Gamma discount factor.
        sampling_function: Function that generates data from one episode.
    
    Returns:
        A dictionary that maps from state -> value.
        The state is a tuple and the value is a float.
    """

    # Keeps track of current V and count of returns for each state
    # to calculate an update.
    V = defaultdict(float)
    returns_count = defaultdict(float)
    
    # YOUR CODE HERE
    
    epsilon = 1e-6
    
        
    # Due to the structure of the gym environment, it is not trivial to map the entire state space
    # so we only map the state space of the BlackJack env
    count_zeros = False
    if (isinstance(env.observation_space, gym.spaces.tuple_space.Tuple)):
        if (len(env.observation_space.spaces) == 3):
            count_zeros = True
        
    state_tuples = [(first, second, bool(third)) for first in range(2,env.observation_space.spaces[0].n)
                       for second in range(1,env.observation_space.spaces[1].n)
                       for third in range(env.observation_space.spaces[2].n)]
    returns = {state_tuple: [] for state_tuple in state_tuples}
    
    if count_zeros:
        returns_count = Counter({state_tuple: 0 for state_tuple in state_tuples})
    
    for episode in tqdm(range(num_episodes)): # num_episodes
        
        env.reset()
        states, actions, rewards, dones = sampling_function(env, behavior_policy)
        p_return = 0
        
        pi = target_policy.get_probs(states, actions)
        b = (behavior_policy.get_probs(states, actions) + epsilon)
        pi_div_b = target_policy.get_probs(states, actions) / (behavior_policy.get_probs(states, actions) + epsilon)

        for index in reversed(range(len(states))): # Reverse so we loop in opposite direction through timesteps
            c_state = states[index]
            c_action = actions[index]
            c_reward = rewards[index]

            p_return = discount_factor * p_return + c_reward
            W = np.cumprod(pi_div_b[index:])
            
            p_return = W[0] * p_return
            if len(returns[c_state]) == 0:
                returns[c_state] = [p_return]
            else:
                returns[c_state].append(p_return)

            if count_zeros:
                returns_count[c_state] += 1
    
    V = {state: np.nan_to_num(np.mean(value)) for (state, value) in returns.items()}
    
    if count_zeros:
        zero_counts = [True for item in list(returns_count) if returns_count[item] == 0]
        no_of_zero = sum(zero_counts)
        if no_of_zero>0:
            print(f"Did not reach {no_of_zero} states in MC estimation. Value estimation for these states is missing.")
        else:
            print("Reached all states in MC estimation.")
    
    return V

In [ ]:
V_10k = mc_importance_sampling(env, RandomBlackjackPolicy(), SimpleBlackjackPolicy(), num_episodes=10000)
print(V_10k)

In [ ]:
%%time
# Let's run your code one time
V_10k = mc_importance_sampling(env, RandomBlackjackPolicy(), SimpleBlackjackPolicy(), num_episodes=10000)
V_500k = mc_importance_sampling(env, RandomBlackjackPolicy(), SimpleBlackjackPolicy(), num_episodes=500000)

Plot the V function. Do the plots look like what you expected?

In [ ]:
# YOUR CODE HERE
for iteration_index, V_dict in enumerate([V_10k, V_500k]):
    for usable_index, usable in enumerate([True, False]):
        ax = plt.subplot(2, 2, 1+iteration_index + 2*usable_index, projection='3d')
        X, Y, Z, ax = get_mesh(V_dict, usable=usable, plot_surf=True, ax=ax)
        ax.set_zlim3d(-1, 2)
        if usable:
            if iteration_index == 0:
                ax.set_title("Usable ace at 10k episodes")
            else:
                ax.set_title("Usable ace at 500k episodes")
        else:
            if iteration_index == 0:
                ax.set_title("No usable ace at 10k episodes")
            else:
                ax.set_title("No usable ace at 500k episodes")
        
# The value function is quite different, since it is now between 0 and 2 rather than -1 and 1. Additionally, 
# the states with a low value in the previous evaluation are less noisy now - the low value is flat at 0.
# The value function is also less steep.

If you want to test/submit your solution **restart the kernel, run all cells and submit the mc_autograde.py file into codegrade.**